# DREAMER Dominance FastGRNN 48_16

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
from sklearn.model_selection import train_test_split
import pathlib
from os import mkdir

In [2]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val
def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))
def bagData(X, Y, subinstanceLen, subinstanceStride):
    numClass = 5
    numSteps = 128
    numFeats = 16
    assert X.ndim == 3
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0
    assert subinstanceStride <= numSteps
    assert subinstanceStride >= 0
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

In [4]:
mkdir('/home/sf/data/DREAMER/Dominance/Fast_GRNN')

In [5]:
subinstanceLen=48
subinstanceStride=16
extractedDir = '/home/sf/data/DREAMER/Dominance/'
mkdir('/home/sf/data/DREAMER/Dominance/Fast_GRNN/48_16')
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + 'Fast_GRNN' '/%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

Num train 61735
Num test 17149
Num val 6860


In [6]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# FastGRNN and FastRNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_FastGRNN
from edgeml.graph.rnn import EMI_FastRNN
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [8]:
# Network parameters for our FastGRNN + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 48
NUM_FEATS = 16
FORGET_BIAS = 1.0
NUM_OUTPUT = 5
USE_DROPOUT = False
KEEP_PROB = 0.9

# Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"

# Ranks of Parameter matrices for low-rank parameterisation to compress models.
WRANK = 5
URANK = 6

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 2

# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4

# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 10

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn'

# Loading Data

In [ ]:
# Loading the data
path='/home/sf/data/DREAMER/Dominance/Fast_GRNN/48_16/'
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

# Computation Graph

In [10]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict

    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
if USE_DROPOUT is True:
    EMI_FastGRNN.feedDictFunc = feedDictFunc

In [11]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiFastGRNN = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                           gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

In [12]:
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (61735, 6, 48, 16)
y_train shape is: (61735, 6, 5)
x_test shape is: (6860, 6, 48, 16)
y_test shape is: (6860, 6, 5)


In [13]:
tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    # Obtain the iterators to each batch of the data
    x_batch, y_batch = inputPipeline()
    # Create the forward computation graph based on the iterators
    y_cap = emiFastGRNN(x_batch)
    # Create loss graphs and training routines
    emiTrainer(y_cap, y_batch)

# EMI Driver

In [14]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiFastGRNN, emiTrainer)

emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

Update policy: top-k
Training with MI-RNN loss for 5 rounds
Round: 0
Epoch   1 Batch  1925 ( 3855) Loss 0.02630 Acc 0.47396 | Val acc 0.39242 | Model saved to /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn, global_step 1000
Epoch   1 Batch  1925 ( 3855) Loss 0.02545 Acc 0.47917 | Val acc 0.40262 | Model saved to /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn, global_step 1001
Epoch   1 Batch  1925 ( 3855) Loss 0.02509 Acc 0.47917 | Val acc 0.41399 | Model saved to /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn, global_step 1002
Epoch   1 Batch  1925 ( 3855) Loss 0.02476 Acc 0.53125 | Val acc 0.42274 | Model saved to /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn, global_step 1003
INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1003
Round: 1
Epoch   1 Batch  1925 ( 3855) Loss 0.02451 Acc 0.55208 | Val acc 0.43921 | Model saved to /home/sf/data/DREA

INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1039


# Evaluating the  trained model

In [15]:
# Early Prediction Policy: We make an early prediction based on the predicted classes
#     probability. If the predicted class probability > minProb at some step, we make
#     a prediction at that step.
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings

In [16]:
k = 2
ORIGINAL_NUM_TIMESTEPS = 128
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Test Accuracy (k = %d): %f\n' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
total_savings = mi_savings + (1 - mi_savings) * emi_savings
print("Total Savings: %f" % total_savings)

Test Accuracy (k = 2): 0.528194

Total Savings: 0.629757


In [17]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.529535   0.448461   0.626394   0.424688   0.529535   0.529535   
1    2  0.528194   0.454806   0.504050   0.435102   0.528194   0.528194   
2    3  0.481777   0.417933   0.478811   0.430978   0.481777   0.481777   
3    4  0.396058   0.375566   0.522220   0.401936   0.396058   0.396058   
4    5  0.336055   0.343254   0.570104   0.381710   0.336055   0.336055   
5    6  0.292437   0.315475   0.611050   0.365957   0.292437   0.292437   

   micro-rec  
0   0.529535  
1   0.528194  
2   0.481777  
3   0.396058  
4   0.336055  
5   0.292437  
Max accuracy 0.529535 at subsequencelength 1
Max micro-f 0.529535 at subsequencelength 1
Micro-precision 0.529535 at subsequencelength 1
Micro-recall 0.529535 at subsequencelength 1
Max macro-f 0.454806 at subsequencelength 2
macro-precision 0.504050 at subsequencelength 2
macro-recall 0.435102 at subsequencelength 2


## Picking the best model

In [18]:
devnull = open(os.devnull, 'r')
for val in modelStats:
    round_, acc, modelPrefix, globalStep = val
    emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
 
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
    print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
    print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 

INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1003
Round:  0, Validation accuracy: 0.4227, Test Accuracy (k = 2): 0.425622, Additional savings: 0.000027
INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1007
Round:  1, Validation accuracy: 0.4776, Test Accuracy (k = 2): 0.477346, Additional savings: 0.001708
INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1011
Round:  2, Validation accuracy: 0.5013, Test Accuracy (k = 2): 0.499738, Additional savings: 0.003691
INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1014
Round:  3, Validation accuracy: 0.5166, Test Accuracy (k = 2): 0.507377, Additional savings: 0.004199
INFO:tensorflow:Restoring parameters from /home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn-1018
Round:  4, Validation 

In [19]:
# Network parameters for our FastGRNN + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 48
NUM_FEATS = 16
FORGET_BIAS = 1.0
NUM_OUTPUT = 5
USE_DROPOUT = False
KEEP_PROB = 0.9

# Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"

# Ranks of Parameter matrices for low-rank parameterisation to compress models.
WRANK = 5
URANK = 6

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 2
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 10

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn'

In [20]:
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 48, #subinstance length.
    "ORIGINAL_NUM_TIMESTEPS" : 128,
    "NUM_FEATS" : 16,
    "FORGET_BIAS" : 1.0,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "NUM_OUTPUT" : 5,
    "WRANK" : 5,
    "URANK" : 6,
    "USE_DROPOUT" : 0, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.9,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "MODEL_PREFIX" : '/home/sf/data/DREAMER/Dominance/48_16/models/Fast-GRNN/model-fgrnn'
}

In [21]:
fgrnn_dict = {**params}
fgrnn_dict["k"] = k
fgrnn_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
fgrnn_dict["total_savings"] = getEarlySaving(predictionStep, NUM_TIMESTEPS)
fgrnn_dict["y_test"] = BAG_TEST
fgrnn_dict["y_pred"] = bagPredictions

# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
print (tabulate(df, headers=list(df.columns), tablefmt='grid'))

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.529535   0.448461   0.626394   0.424688   0.529535   0.529535   
1    2  0.528194   0.454806   0.504050   0.435102   0.528194   0.528194   
2    3  0.481777   0.417933   0.478811   0.430978   0.481777   0.481777   
3    4  0.396058   0.375566   0.522220   0.401936   0.396058   0.396058   
4    5  0.336055   0.343254   0.570104   0.381710   0.336055   0.336055   
5    6  0.292437   0.315475   0.611050   0.365957   0.292437   0.292437   

   micro-rec  
0   0.529535  
1   0.528194  
2   0.481777  
3   0.396058  
4   0.336055  
5   0.292437  
Max accuracy 0.529535 at subsequencelength 1
Max micro-f 0.529535 at subsequencelength 1
Micro-precision 0.529535 at subsequencelength 1
Micro-recall 0.529535 at subsequencelength 1
Max macro-f 0.454806 at subsequencelength 2
macro-precision 0.504050 at subsequencelength 2
macro-recall 0.435102 at subsequencelength 2
+----+-------+----------+-------------+-----------

In [22]:
dirname = "/home/sf/data/DREAMER/Dominance/Fast_GRNN/"
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(fgrnn_dict,open(dirname  + filename + ".pkl",mode='wb'))

Results for this run have been saved at /home/sf/data/DREAMER/Dominance/Fast_GRNN/ .
